In [133]:
import pandas as pd
from unidecode import unidecode
import re
from sqlalchemy import create_engine


In [134]:
dados = pd.read_csv("../0_bases_originais/Placa_de_Video.csv", sep=";")

In [135]:
dados

,Produtos,Valor,valor parcelado
0,Placa de Vídeo RTX 4060 Ti OC Edition Asus TUF...,"R$ 2.899,99","10x de R$ 341,17"
1,Placa de Vídeo RTX 4060 8G V2 Gaming ATS OC AS...,"R$ 2.199,99","10x de R$ 258,82"
2,Placa de Vídeo RTX 3060 1-Click OC Galax NVIDI...,"R$ 1.639,99","10x de R$ 192,94"
3,Placa de Vídeo RX 6600 8G-V3 Dual ASUS AMD Rad...,"R$ 1.299,99","10x de R$ 152,94"
4,Placa de Vídeo RX 6750XT Gaming XFX Graphics C...,"R$ 2.099,99","10x de R$ 247,05"
...,...,...,...
695,Placa de Vídeo Gigabyte NVIDIA GeForce RTX 306...,NaN,NaN
696,Placa de Vídeo Gigabyte NVIDIA GeForce RTX 306...,NaN,NaN
697,Placa de Vídeo Gigabyte NVIDIA GeForce RTX 309...,NaN,NaN
698,Placa de Vídeo Gigabyte NVIDIA AORUS GeForce R...,NaN,NaN


In [136]:
df = dados.dropna()
df = df.drop_duplicates()

In [137]:
df2 = df.reset_index(drop=True)

In [138]:
df2['Valor'] = df2['Valor'].str.replace('R$','').str.replace(',','').str.replace('.','')
df2['valor parcelado'] = df2['valor parcelado'].str.replace('R$','').str.replace(',','').str.replace('.','').str.replace('de','').str.replace(' ','')
df2['Valor'] = df2['Valor'].astype(float)
df2[['parcela','valorparcela']] = df2['valor parcelado'].str.split('x', expand=True)
df2['valorparcela'] = df2['valorparcela'].astype(float)
df2['parcela'] = df2['parcela'].astype(float)
df2['valor parcelado'] = df2['valorparcela'] * df2['parcela']
df2[['Valor','valor parcelado']] = df2[['Valor','valor parcelado']] /100
df2 = df2.drop(columns=['parcela','valorparcela'], axis=1)

In [139]:
df2.dtypes

Produtos            object
Valor              float64
valor parcelado    float64
dtype: object

In [140]:
Q1 = df2['Valor'].quantile(0.75)
Q3 = df2['Valor'].quantile(0.25)
Q4 = df2['valor parcelado'].quantile(0.75)
Q5 = df2['valor parcelado'].quantile(0.25)

df2 = df2[(df2['Valor'] >= Q3) & (df2['Valor'] <= Q1)]
df2 = df2[(df2['valor parcelado'] >= Q5) & (df2['valor parcelado'] <= Q4)]

In [141]:
df2['Produtos'] = df2['Produtos'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', '', unidecode(x)))

In [ ]:
df2.to_csv('../1_bases_tratadas/Placa_de_Video.csv', sep = ';', index=False)

NameError: name 'df2' is not defined

In [142]:
engine = create_engine('sqlite:///banco.db', echo = True)
df2.to_sql('Dados', con  = engine, if_exists='replace', index=False)



2024-11-22 16:58:18,762 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-22 16:58:18,763 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Dados")
2024-11-22 16:58:18,764 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-22 16:58:18,764 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Dados")
2024-11-22 16:58:18,764 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-22 16:58:18,765 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-11-22 16:58:18,765 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-22 16:58:18,766 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-11-22 16:58:18,767 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-22 16:58:18,771 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Dados")
2024-11-22 16:58:18,772 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-22 16:58:1

282